# Customer Cohort & Lifetime Value Analysis

**Purpose:** Deep dive into customer cohorts, retention patterns, and lifetime value calculations.

**Key Questions:**
- Which cohorts have the highest retention?
- What is the estimated customer lifetime value by segment?
- When do customers typically churn?
- Which acquisition months produced the best customers?

**Data Sources:**
- `mart_customer_cohorts` - Cohort composition and metrics
- `mart_customer_retention` - Retention curves over time

---

## Setup

Import libraries and initialize notebook environment.

In [ ]:
# Import libraries
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add current directory to path for importing notebook_utils
notebook_dir = Path().absolute()
if str(notebook_dir) not in sys.path:
    sys.path.insert(0, str(notebook_dir))

# Import utilities
from notebook_utils import (
    DataValidator,
    VisualizationHelper,
    export_dataframe,
    setup_notebook_environment,
)

# Setup environment
config, client, bq_helper = setup_notebook_environment()
viz = VisualizationHelper(figsize=(14, 8))

print("✅ Setup complete - Ready for cohort analysis")

## 1. Cohort Overview

Load and understand cohort composition.

In [ ]:
# Query: Cohort data
query_cohorts = f"""
SELECT
  cohort_month,
  cohort_month_name,
  customer_state,
  cohort_size,
  loyal_customers,
  repeat_customers,
  one_time_customers,
  customer_ltv,
  avg_orders_per_customer,
  total_revenue,
  avg_review_score,
  loyal_pct,
  repeat_pct,
  one_time_pct
FROM `{config.project_id}.{config.dataset}.mart_customer_cohorts`
WHERE cohort_month >= '2017-01-01'
ORDER BY cohort_month, customer_state
"""

# Execute query
df_cohorts = bq_helper.query_to_dataframe(query_cohorts)
DataValidator.check_dataframe_not_empty(df_cohorts, "Cohort Data")
DataValidator.check_columns_exist(
    df_cohorts, ["cohort_month", "cohort_size", "customer_ltv"], "Cohort Data"
)

# Display overview
print(f"📊 Loaded {len(df_cohorts)} cohort records")
print(f"📅 Date Range: {df_cohorts['cohort_month'].min()} to {df_cohorts['cohort_month'].max()}")
print(f"👥 Total Customers: {df_cohorts['cohort_size'].sum():,}")

## 2. Lifetime Value by Cohort

Compare customer lifetime value across acquisition cohorts.

In [ ]:
# Aggregate by cohort month (across all states)
cohort_summary = (
    df_cohorts.groupby("cohort_month_name")
    .agg(
        {
            "cohort_size": "sum",
            "customer_ltv": "mean",
            "avg_orders_per_customer": "mean",
            "total_revenue": "sum",
            "avg_review_score": "mean",
        }
    )
    .reset_index()
)

# Create visualizations
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# LTV trend
ax1.bar(
    cohort_summary["cohort_month_name"],
    cohort_summary["customer_ltv"],
    color=viz.colors["success"],
    alpha=0.7,
)
ax1.axhline(
    y=cohort_summary["customer_ltv"].mean(),
    color="red",
    linestyle="--",
    label=f"Average: R${cohort_summary['customer_ltv'].mean():.2f}",
)
ax1.set_title("Customer Lifetime Value by Acquisition Cohort", fontsize=16, fontweight="bold")
ax1.set_ylabel("LTV (R$)", fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3, axis="y")
ax1.tick_params(axis="x", rotation=45)

# Cohort size trend
ax2.plot(
    cohort_summary["cohort_month_name"],
    cohort_summary["cohort_size"],
    marker="o",
    linewidth=2,
    color=viz.colors["primary"],
)
ax2.set_title("Cohort Size Over Time", fontsize=16, fontweight="bold")
ax2.set_ylabel("Customers", fontsize=12)
ax2.set_xlabel("Cohort Month", fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

# Display top cohorts
print("\n🏆 Top 5 Cohorts by LTV")
top_ltv = cohort_summary.nlargest(5, "customer_ltv")[
    ["cohort_month_name", "customer_ltv", "cohort_size"]
]
print(top_ltv.to_string(index=False))

## 3. Segment Distribution by Cohort

Analyze how customer segment distribution varies across cohorts.

In [ ]:
# Calculate segment percentages by cohort
cohort_summary["loyal_pct"] = df_cohorts.groupby("cohort_month_name")["loyal_pct"].mean().values
cohort_summary["repeat_pct"] = df_cohorts.groupby("cohort_month_name")["repeat_pct"].mean().values
cohort_summary["one_time_pct"] = (
    df_cohorts.groupby("cohort_month_name")["one_time_pct"].mean().values
)

# Create stacked bar chart
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(cohort_summary))
width = 0.8

p1 = ax.bar(
    x, cohort_summary["loyal_pct"], width, label="Loyal (3+ orders)", color=viz.colors["success"]
)
p2 = ax.bar(
    x,
    cohort_summary["repeat_pct"],
    width,
    bottom=cohort_summary["loyal_pct"],
    label="Repeat (2 orders)",
    color=viz.colors["warning"],
)
p3 = ax.bar(
    x,
    cohort_summary["one_time_pct"],
    width,
    bottom=cohort_summary["loyal_pct"] + cohort_summary["repeat_pct"],
    label="One-time (1 order)",
    color=viz.colors["danger"],
)

ax.set_ylabel("Percentage", fontsize=12)
ax.set_title("Customer Segment Distribution by Cohort", fontsize=16, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(cohort_summary["cohort_month_name"], rotation=45, ha="right")
ax.legend()
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

# Display insights
print("\n💡 Segment Insights")
print(f"  • Average Loyal: {cohort_summary['loyal_pct'].mean():.1f}%")
print(f"  • Average Repeat: {cohort_summary['repeat_pct'].mean():.1f}%")
print(f"  • Average One-time: {cohort_summary['one_time_pct'].mean():.1f}%")

## 4. Retention Curves

Analyze how different cohorts retain customers over time.

In [ ]:
# Query: Retention curves for first 6 cohorts
query_retention = f"""
SELECT
  cohort_month_name,
  months_since_cohort,
  retention_rate_pct,
  active_customers,
  cohort_size
FROM `{config.project_id}.{config.dataset}.mart_customer_retention`
WHERE cohort_month >= '2017-01-01'
  AND cohort_month <= '2017-06-01'
  AND months_since_cohort <= 12
ORDER BY cohort_month, months_since_cohort
"""

# Execute query
df_retention = bq_helper.query_to_dataframe(query_retention)
DataValidator.check_dataframe_not_empty(df_retention, "Retention Data")

# Plot retention curves
fig, ax = plt.subplots(figsize=(14, 8))

for cohort in df_retention["cohort_month_name"].unique():
    cohort_data = df_retention[df_retention["cohort_month_name"] == cohort]
    ax.plot(
        cohort_data["months_since_cohort"],
        cohort_data["retention_rate_pct"],
        marker="o",
        linewidth=2,
        label=cohort,
    )

ax.set_xlabel("Months Since First Order", fontsize=12)
ax.set_ylabel("Retention Rate (%)", fontsize=12)
ax.set_title("Customer Retention Curves by Cohort", fontsize=16, fontweight="bold")
ax.legend(title="Cohort", bbox_to_anchor=(1.05, 1), loc="upper left")
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()

# Calculate average retention
avg_retention = df_retention.groupby("months_since_cohort")["retention_rate_pct"].mean()
print("\n📊 Average Retention Rates")
for month in [1, 3, 6, 12]:
    if month in avg_retention.index:
        print(f"  • Month {month}: {avg_retention[month]:.1f}%")

## 5. Revenue by Cohort Age

Analyze cumulative revenue contribution over time.

In [ ]:
# Query: Cumulative revenue by cohort
query_revenue = f"""
SELECT
  cohort_month_name,
  months_since_cohort,
  cumulative_revenue,
  cohort_size
FROM `{config.project_id}.{config.dataset}.mart_customer_retention`
WHERE cohort_month >= '2017-01-01'
  AND cohort_month <= '2017-03-01'
  AND months_since_cohort <= 12
ORDER BY cohort_month, months_since_cohort
"""

# Execute query
df_revenue = bq_helper.query_to_dataframe(query_revenue)
DataValidator.check_dataframe_not_empty(df_revenue, "Revenue Data")

# Calculate revenue per customer
df_revenue["revenue_per_customer"] = df_revenue["cumulative_revenue"] / df_revenue["cohort_size"]

# Create dual charts
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Cumulative revenue
for cohort in df_revenue["cohort_month_name"].unique():
    cohort_data = df_revenue[df_revenue["cohort_month_name"] == cohort]
    ax1.plot(
        cohort_data["months_since_cohort"],
        cohort_data["cumulative_revenue"],
        marker="o",
        linewidth=2,
        label=cohort,
    )

ax1.set_xlabel("Months Since First Order", fontsize=12)
ax1.set_ylabel("Cumulative Revenue (R$)", fontsize=12)
ax1.set_title("Cumulative Revenue by Cohort", fontsize=14, fontweight="bold")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Revenue per customer
for cohort in df_revenue["cohort_month_name"].unique():
    cohort_data = df_revenue[df_revenue["cohort_month_name"] == cohort]
    ax2.plot(
        cohort_data["months_since_cohort"],
        cohort_data["revenue_per_customer"],
        marker="o",
        linewidth=2,
        label=cohort,
    )

ax2.set_xlabel("Months Since First Order", fontsize=12)
ax2.set_ylabel("Revenue per Customer (R$)", fontsize=12)
ax2.set_title("Revenue per Customer by Cohort", fontsize=14, fontweight="bold")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Actionable Recommendations

Generate data-driven recommendations based on cohort analysis.

In [ ]:
print("\n" + "=" * 80)
print("🎯 ACTIONABLE RECOMMENDATIONS")
print("=" * 80)

# 1. Acquisition Strategy
best_ltv_cohort = cohort_summary.loc[cohort_summary["customer_ltv"].idxmax()]
print("\n1️⃣ ACQUISITION STRATEGY")
print(
    f"   • Best cohort: {best_ltv_cohort['cohort_month_name']} (LTV: R${best_ltv_cohort['customer_ltv']:.2f})"
)
print("   • Recommendation: Analyze marketing campaigns from this period")
print("   • Action: Replicate successful acquisition channels and messaging")

# 2. Retention Improvement
avg_loyal = cohort_summary["loyal_pct"].mean()
print("\n2️⃣ RETENTION IMPROVEMENT")
print(f"   • Current loyal customer rate: {avg_loyal:.1f}%")
if avg_loyal < 30:
    print("   • Target: Increase to 30% through loyalty programs")
    print("   • Action: Implement points/rewards system for repeat purchases")
else:
    print("   • Action: Maintain current retention programs")

# 3. Churn Prevention
if "avg_retention" in locals():
    print("\n3️⃣ CHURN PREVENTION")
    if 3 in avg_retention.index and avg_retention[3] < 50:
        print(f"   • Critical drop at month 3: {avg_retention[3]:.1f}% retention")
        print("   • Action: Launch win-back campaign at 60-90 days post-purchase")
        print("   • Tactic: Personalized email with discount code")

# 4. Segment Focus
one_time_pct = cohort_summary["one_time_pct"].mean()
print("\n4️⃣ SEGMENT FOCUS")
if one_time_pct > 50:
    print(f"   • {one_time_pct:.1f}% are one-time customers")
    print("   • Action: Create 'second purchase' incentive campaign")
    print("   • Target: Reduce one-time customers to <40%")

# 5. LTV Optimization
avg_ltv = cohort_summary["customer_ltv"].mean()
print("\n5️⃣ LTV OPTIMIZATION")
print(f"   • Current average LTV: R${avg_ltv:.2f}")
print(f"   • Potential with 20% improvement: R${avg_ltv * 1.2:.2f}")
print("   • Action: Upsell/cross-sell campaigns for repeat customers")
print("   • Focus: Product bundling and category expansion")

print("\n" + "=" * 80)
print("✅ Analysis Complete - Ready for Stakeholder Presentation")
print("=" * 80)